# Working with parquet files

## Objective

+ In this assignment, we will use the data downloaded with the module `data_manager` to create features.

(11 pts total)

## Prerequisites

+ This notebook assumes that price data is available to you in the environment variable `PRICE_DATA`. If you have not done so, then execute the notebook `01_materials/labs/2_data_engineering.ipynb` to create this data set.


+ Load the environment variables using dotenv. (1 pt)

In [4]:
# Write your code below.

%load_ext dotenv
%dotenv 

The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv


In [5]:
import dask.dataframe as dd

+ Load the environment variable `PRICE_DATA`.
+ Use [glob](https://docs.python.org/3/library/glob.html) to find the path of all parquet files in the directory `PRICE_DATA`.

(1pt)

In [11]:
import os
from glob import glob

# Write your code below.

price_data_path = os.getenv("PRICE_DATA")
os.path.join(price_data_path, "**/part.0.parquet")
parquet_files = glob(os.path.join(price_data_path, "**/part.0.parquet"), recursive = True)
dd_feat=dd.read_parquet(parquet_files)


For each ticker and using Dask, do the following:

+ Add lags for variables Close and Adj_Close.
+ Add returns based on Adjusted Close:
    
    - `returns`: (Adj Close / Adj Close_lag) - 1

+ Add the following range: 

    - `hi_lo_range`: this is the day's High minus Low.

+ Assign the result to `dd_feat`.

(4 pt)

In [12]:
# Write your code below.
dd_feat = dd_feat.groupby('ticker', group_keys=False).apply(
    lambda x: x.assign(
        Close_lag=x['Close'].shift(1),
        Adj_Close_lag=x['Adj Close'].shift(1)
    )
)

dd_feat['returns'] = (dd_feat['Adj Close'] / dd_feat['Adj_Close_lag']) - 1
dd_feat['hi_lo_range'] = dd_feat['High'] - dd_feat['Low']


C:\Users\alyon\AppData\Local\Temp\ipykernel_17324\3445063403.py:2: UserWarning: `meta` is not specified, inferred from partial data. Please provide `meta` if the result is unexpected.
  Before: .apply(func)
  After:  .apply(func, meta={'x': 'f8', 'y': 'f8'}) for dataframe result
  or:     .apply(func, meta=('x', 'f8'))            for series result
  dd_feat = dd_feat.groupby('ticker', group_keys=False).apply(


+ Convert the Dask data frame to a pandas data frame. 
+ Add a rolling average return calculation with a window of 10 days.
+ *Tip*: Consider using `.rolling(10).mean()`.

(3 pt)

In [13]:
# Write your code below.
import pandas as pd

dd_feat = dd_feat.compute()
dd_feat = dd_feat.reset_index(level='ticker')  
dd_feat['rolling_avg_return'] = dd_feat.groupby('ticker')['returns'].transform(lambda x: x.rolling(10).mean())
dd_feat.head()


,ticker,Date,Open,High,Low,Close,Adj Close,Volume,sector,subsector,year,Close_lag,Adj_Close_lag,returns,hi_lo_range,rolling_avg_return
0,SYY,2006-01-03,31.209999,31.230000,30.809999,31.090000,17.966312,2118300,Consumer Staples,Food Distributors,2006,NaN,NaN,NaN,0.420000,NaN
1,SYY,2006-01-04,31.049999,31.480000,31.030001,31.110001,18.076721,2286200,Consumer Staples,Food Distributors,2006,31.090000,17.966312,0.006145,0.449999,NaN
2,SYY,2006-01-05,31.120001,31.340000,30.889999,30.990000,18.006990,1847600,Consumer Staples,Food Distributors,2006,31.110001,18.076721,-0.003857,0.450001,NaN
3,SYY,2006-01-06,30.990000,31.360001,30.879999,31.250000,18.158068,1905500,Consumer Staples,Food Distributors,2006,30.990000,18.006990,0.008390,0.480001,NaN
4,SYY,2006-01-09,31.299999,31.549999,31.200001,31.389999,18.239412,1738000,Consumer Staples,Food Distributors,2006,31.250000,18.158068,0.004480,0.349998,NaN


Please comment:

+ Was it necessary to convert to pandas to calculate the moving average return?

Not necessary because the Dask can perform this itself

+ Would it have been better to do it in Dask? Why?

to save time first, dask can better performm with big datasets
(1 pt)

## Criteria

The [rubric](./assignment_1_rubric_clean.xlsx) contains the criteria for grading.

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-1`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_1.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.